In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

/home/g/10/week6/Intelligent-Complaint-Analysis-for-Financial-Services/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the FAISS index and metadata
index = faiss.read_index('vector_store/faiss_index.index')
metadata_df = pd.read_csv('vector_store/metadata.csv')

In [4]:
# Load the embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B')  # Alternative model
# Initialize the language model for text generation
#generator = pipeline('text-generation', model='gpt-3.5-turbo')  # Ensure you have access
def retrieve_relevant_chunks(question, k=5):
    # Embed the user's question
    question_embedding = embedding_model.encode([question])
    # Perform similarity search in the FAISS index
    distances, indices = index.search(np.array(question_embedding, dtype=np.float32), k)
    
    # Retrieve the corresponding chunks and metadata
    retrieved_chunks = []
    for idx in indices[0]:
        if idx != -1:  # Ensure the index is valid
            retrieved_chunks.append(metadata_df.iloc[idx]['original_id'])
    
    return retrieved_chunks

Device set to use cpu


In [7]:
def generate_answer(question, context):
    # Define the prompt template
    prompt_template = (
        "You are a financial analyst assistant for CrediTrust. "
        "Your task is to answer questions about customer complaints. "
        "Use the following retrieved complaint excerpts to formulate your answer. "
        "If the context doesn't contain the answer, state that you don't have enough information. "
        "Context: {context} "
        "Question: {question} "
        "Answer:"
    )
    
    # Ensure all context items are strings
    context_str = "\n".join(str(chunk) for chunk in context)
    
    # Generate the final prompt
    prompt = prompt_template.format(context=context_str, question=question)
    
    # Get the generated response from the model
    response = generator(prompt, max_length=150)
    return response[0]['generated_text']

# Example usage
if __name__ == "__main__":
    sample_question = "What issues do customers face with credit cards?"
    relevant_chunks = retrieve_relevant_chunks(sample_question)
    generated_answer = generate_answer(sample_question, relevant_chunks)
    print("Generated Answer:", generated_answer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Answer: You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints. Use the following retrieved complaint excerpts to formulate your answer. If the context doesn't contain the answer, state that you don't have enough information. Context: 2384261
10275768
12507997
9455030
10147009 Question: What issues do customers face with credit cards? Answer: They have long processing times and high credit card fees. The complaint does not identify exactly which issues customers are facing, but that they have a lot of complaints about their credit cards.
10923192
11472041
10653555
10443984
10653420
10127896 Question: What is the source of a complaint, and where can I find more information on it? Answer: The complaint is most likely from the customer. When a customer makes a complaint, CrediTrust tries to get in touch with the person. If you do not have enough information to answer that question, state that. Context: 12471138
11472202
1